# Adding Back the Tides - CIOPS
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

In [7]:
startday = [dt.datetime(2015,11,22)+dt.timedelta(days=14*i) for i in range(int(214))]
print(startday[-1])

2024-01-21 00:00:00


In [8]:
start = dt.datetime(2015,11,22) #start day of your run

#dates for each run
numdays = 8 #15 for all except the last run (7) 
date_list = [start + dt.timedelta(days=x) for x in range(numdays)]
folderday = [start+ dt.timedelta(days=7),start+ dt.timedelta(days=7*2)]#,start+ dt.timedelta(days=7*3)]
folderday = np.repeat(folderday,7)
folderday = folderday[:-6]

In [10]:
date_list

[datetime.datetime(2015, 11, 22, 0, 0),
 datetime.datetime(2015, 11, 23, 0, 0),
 datetime.datetime(2015, 11, 24, 0, 0),
 datetime.datetime(2015, 11, 25, 0, 0),
 datetime.datetime(2015, 11, 26, 0, 0),
 datetime.datetime(2015, 11, 27, 0, 0),
 datetime.datetime(2015, 11, 28, 0, 0),
 datetime.datetime(2015, 11, 29, 0, 0)]

In [11]:
folderday

array([datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 11, 29, 0, 0),
       datetime.datetime(2015, 12, 6, 0, 0)], dtype=object)

In [4]:
"{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[0], date_list[0], date_list[0])

'2015112900/BC12_1h_grid_U_2D_20151122_20151122.nc'

In [5]:
path = Path("/ocean/mdunphy/CIOPSW-BC12/")

drop_vars = (
    "sbu", "tauuo", "time_counter_bounds", "time_instant_bounds", "uos",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
ut_h = mydata['ubar']

In [6]:
ut_h

<xarray.DataArray 'ubar' (time_counter: 360, y: 1020, x: 714)>
dask.array<concatenate, shape=(360, 1020, 714), dtype=float32, chunksize=(24, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
  * time_counter  (time_counter) datetime64[ns] 2015-11-22T01:00:00 ... 2015-...
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
Dimensions without coordinates: y, x
Attributes:
    long_name:           ocean barotropic current along i-axis
    units:               m/s
    online_operation:    instant
    interval_operation:  60 s
    interval_write:      1 h
    cell_methods:        time: point (interval: 60 s)

In [5]:
drop_vars = (
    "sbv", "tauvo", "time_counter_bounds", "time_instant_bounds", "vos",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_V_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
vt_h = mydata['vbar']

In [6]:
# 2D hourly outputs are alraedy in bartropic form so you dont need to do any conversion!

In [7]:
# Now get the required data from the daily files
drop_vars = (
    "depthu_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_U_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['uo']

drop_vars = (
    "depthv_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_V_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vo']

In [8]:
# we WILL need to do some conversions here, so get e3t from the mesh_mask file
mydata = xr.open_dataset("/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc")
e3t = mydata['e3t_0']
e3t

<xarray.DataArray 'e3t_0' (t: 1, z: 75, y: 1020, x: 714)>
[54621000 values with dtype=float64]
Dimensions without coordinates: t, z, y, x

In [10]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5
e3u = e3u.rename({'z': 'depthu'})

e3t_yshift = e3t.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'z': 'depthv'})

In [12]:
#calcuate bartropic component
ut_d = (u_d*e3u[0,:,:,:]).sum(dim='depthu')/e3u[0,:,:,:].sum(dim='depthu')

In [13]:
#subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?

interpolate + resample uc_d to get it in an hourly format

In [30]:
offset = dt.timedelta(hours=-23) # daily avg timestamp as next day time 0, offset by -23 hours to line up with hourly data
uc_h_interp = uc_d.resample(time_counter="1H", loffset=offset).interpolate("linear")
#instead of taking 12hours off each side like when done with SSC, this method takes off 24hours on the end!

In [28]:
u_new = ut_h  + uc_h_interp

In [29]:
u_new

<xarray.DataArray (time_counter: 337, y: 1020, x: 714, depthu: 75)>
dask.array<add, shape=(337, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2015-11-22T01:00:00 ... 2015-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [ ]:
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))

In [ ]:
#calcuate bartropic component
vt_d = (v*e3v).sum(dim='depthv')/e3v.sum(dim='depthv')

In [ ]:
#subtract from v to get baroclinic component
vc_d = v-vt_d 

In [ ]:
vc_d.load(scheduler="processes", num_workers=n)

In [ ]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [ ]:
v_new = vt_h  + vc_h_interp

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))